# Typed Objects

In addition to the interfaces offered by `ObjectAPIClient` and `DownloadedObject`, which provides access to Geoscience Objects in a way that is agnostic to the specific object type, the `evo-objects` package also provides a set of "typed objects" that represent specific object types.

As usual, we need to first authenticate, which we can do using the `ServiceManagerWidget` from the `evo-notebooks` package

In [ ]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="your-client-id", cache_location="./notebook-data"
).login()

## Creating a regular 3D grid object

To create a new 'regular-3d-grid' object, we use the `Regular3DGridData` class to define all of the properties of the grid, then pass that to the `Regular3DGrid.create` class method.

The `cell_data` and `vertex_data` properties are dataframes that define the attributes of the cells and vertices respectively. Each column in the dataframe represents a different attribute.

Note the order of the data is important, and must match the order of the cells and vertices in the grid. The cells are ordered such that the x-axis varies the fastest, then y, then z. So for a grid of size (10, 10, 5), the first 10 rows of the dataframe correspond to the cells at (0, 0, 0) to (9, 0, 0), the next 10 rows correspond to (0, 1, 0) to (9, 1, 0), and so on.

In [ ]:
import numpy as np
import pandas as pd

from evo.objects.typed import Point3, Regular3DGrid, Regular3DGridData, Rotation, Size3d, Size3i

grid = Regular3DGridData(
    name="Test Grid",
    origin=Point3(0, 0, 0),
    size=Size3i(10, 10, 5),
    cell_size=Size3d(2.5, 5, 5),
    cell_data=pd.DataFrame(
        {
            "value": np.random.rand(10 * 10 * 5),
            "cat": pd.Categorical.from_codes(np.random.choice(range(4), size=10 * 10 * 5), ["a", "b", "c", "d"]),
        }
    ),
    vertex_data=pd.DataFrame(
        {
            "elevation": np.random.rand(11 * 11 * 6),
        }
    ),
    rotation=Rotation(90, 0, 0),
)
created_grid = await Regular3DGrid.create(manager, grid)
print(f"Created regular-3d-grid object: {created_grid.metadata.url}")

## Downloading a regular 3D grid object

To download an existing 'regular-3d-grid' object, and inspect it, you can use the `Regular3DGrid.from_reference` class method. This method takes the context and the URL of the object to download.

For the example, we will download the object we just created.

In [ ]:
from evo.objects.typed import object_from_reference

grid = await object_from_reference(manager, created_grid.metadata.url)

# Simply displaying the object shows a pretty-printed summary with Portal and Viewer links
grid

### Accessing object properties

You can also access individual properties programmatically:

In [ ]:
print(f"Downloaded regular-3d-grid object with name: {grid.name}")
print(f"Origin: {grid.origin}")
print(f"Size: {grid.size}")
print(f"Cell Size: {grid.cell_size}")
print(f"Rotation: {grid.rotation}")
print(f"Bounding Box: {grid.bounding_box}")

# Portal and Viewer URLs are available as properties
print(f"\nPortal URL: {grid.portal_url}")
print(f"Viewer URL: {grid.viewer_url}")

### Viewing attributes

You can view the available attributes using the `attributes` property, which provides a pretty-printed table:

In [ ]:
# View cell attributes (shortcut for grid.cells.attributes)
grid.attributes

In [ ]:
# View vertex attributes
grid.vertex_attributes

### Getting data as DataFrames

The `to_dataframe()` method provides a convenient way to get the cell data. You can also use `grid.cells.get_dataframe()` and `grid.vertices.get_dataframe()` for more explicit access.

In [ ]:
# Get all cell data using to_dataframe()
await grid.to_dataframe()

In [ ]:
# Or use the explicit cells/vertices access
await grid.vertices.get_dataframe()

## Updating a regular 3D grid object

To update an existing 'regular-3d-grid' object, we can modify the properties of the `Regular3DGrid` instance, and then call the `update` method to submit the changes to the Geoscience Object service. This will create a new version of the object with the updated properties.

Updating the cell or vertex data can be done by calling the `set_dataframe` method on the `cells` or `vertices` attributes respectively. This will upload the new data to the Geoscience Object service, but the object will remain the same version until the `update` method is called.

In [ ]:
print(f"Version before update: {grid.metadata.version_id}")

grid.name = "Updated Test Grid"

await grid.cells.set_dataframe(
    pd.DataFrame(
        {
            "value": np.ones(10 * 10 * 5),
        }
    )
)

# Update the object to create a new version with the modified properties
await grid.update()
print(f"Updated version: {grid.metadata.version_id}")

## Creating a PointSet object

To create a new 'pointset' object, we use the `PointSetData` class to define the point locations and attributes, then pass that to the `PointSet.create` class method.

The `locations` DataFrame must have 'x', 'y', 'z' columns for coordinates. Any additional columns will be treated as point attributes.

In [ ]:
from evo.objects.typed import PointSet, PointSetData

# Create a pointset with 100 random points
num_points = 100
pointset_data = PointSetData(
    name="Test PointSet 1",
    locations=pd.DataFrame(
        {
            "x": np.random.rand(num_points) * 100,
            "y": np.random.rand(num_points) * 100,
            "z": np.random.rand(num_points) * 50,
            "grade": np.random.rand(num_points) * 10,  # Example attribute
            "category": pd.Categorical.from_codes(
                np.random.choice(range(3), size=num_points), ["low", "medium", "high"]
            ),
        }
    ),
)

created_pointset = await PointSet.create(manager, pointset_data)
print(f"Created pointset object: {created_pointset.metadata.url}")

## Downloading a PointSet object

To download an existing 'pointset' object, use the `PointSet.from_reference` class method. Simply displaying the object shows a pretty-printed summary.

In [ ]:
from evo.objects.typed import object_from_reference

pointset = await object_from_reference(manager, created_pointset.metadata.url)

# Pretty-printed display with Portal and Viewer links
pointset

### Accessing pointset properties and data

The `attributes` property provides quick access to point attributes, and `to_dataframe()` returns all data including coordinates.

In [ ]:
print(f"Number of points: {pointset.num_points}")
print(f"Bounding Box: {pointset.bounding_box}")

# View available attributes
pointset.attributes

In [ ]:
# Get the full dataframe with coordinates and attributes using to_dataframe()
await pointset.to_dataframe()

In [ ]:
# Or get just the coordinates
await pointset.coordinates()

## Creating a regular masked 3D grid object

A masked grid is similar to a regular grid, but it includes a boolean mask indicating which cells are "active". Only active cells have attribute values, which can significantly reduce storage when working with sub-regions of a larger grid (e.g., a mining pit or geological domain).

The `mask` is a 1D boolean array with the same length as the total number of cells in the grid. The `cell_data` should only contain rows for the active cells (where `mask=True`).

In [ ]:
from evo.objects.typed import RegularMasked3DGrid, RegularMasked3DGridData

# Create a mask where only some cells are active (e.g., alternating pattern)
mask = np.array([True, False, True, False, True] * 100, dtype=bool)  # 500 cells, 300 active
number_active = np.sum(mask)

masked_grid = RegularMasked3DGridData(
    name="Test Masked Grid 1",
    origin=Point3(0, 0, 0),
    size=Size3i(10, 10, 5),
    cell_size=Size3d(2.5, 5, 5),
    mask=mask,
    cell_data=pd.DataFrame(
        {
            # Only provide data for active cells
            "value": np.random.rand(number_active),
            "category": pd.Categorical.from_codes(
                np.random.choice(range(3), size=number_active), ["rock", "ore", "waste"]
            ),
        }
    ),
    rotation=Rotation(45, 0, 0),
)
created_masked_grid = await RegularMasked3DGrid.create(manager, masked_grid)
print(f"Created regular-masked-3d-grid object: {created_masked_grid.metadata.url}")
print(f"Total cells: {created_masked_grid.size.total_size}, Active cells: {created_masked_grid.cells.number_active}")

### Downloading and inspecting a masked grid

Simply displaying the masked grid shows a pretty-printed summary. The mask can be retrieved using `cells.get_mask()`, which returns a boolean numpy array. The cell data only contains values for active cells.

In [ ]:
from evo.objects.typed import object_from_reference

masked_grid = await object_from_reference(manager, created_masked_grid.metadata.url)

# Pretty-printed display
masked_grid

In [ ]:
# View available attributes
masked_grid.attributes

In [ ]:
print(f"Size: {masked_grid.size}, Active cells: {masked_grid.cells.number_active}")

# Get the mask
grid_mask = await masked_grid.cells.get_mask()
print(f"Mask shape: {grid_mask.shape}, Active count: {np.sum(grid_mask)}")

# Get the cell data using to_dataframe() (only for active cells)
cell_df = await masked_grid.to_dataframe()
print(f"Cell data shape: {cell_df.shape}")
cell_df.head()

## Creating a tensor 3D grid object

A tensor grid is a 3D grid where cells can have different sizes along each axis. Instead of a uniform `cell_size`, you provide arrays of cell sizes for each axis (`cell_sizes_x`, `cell_sizes_y`, `cell_sizes_z`).

This is useful for grids with variable resolution, such as grids that are finer in areas of interest and coarser elsewhere.

In [ ]:
from evo.objects.typed import Tensor3DGrid, Tensor3DGridData

# Define variable cell sizes along each axis
# X-axis: 10 cells with varying sizes (finer in the middle)
cell_sizes_x = np.array([5.0, 3.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 3.0, 5.0])
# Y-axis: 8 cells with uniform sizes
cell_sizes_y = np.array([2.0] * 8)
# Z-axis: 5 cells with increasing sizes (finer at top)
cell_sizes_z = np.array([1.0, 1.5, 2.0, 2.5, 3.0])

size = Size3i(len(cell_sizes_x), len(cell_sizes_y), len(cell_sizes_z))
num_cells = size.total_size
num_vertices = (size.nx + 1) * (size.ny + 1) * (size.nz + 1)

tensor_grid = Tensor3DGridData(
    name="Variable Resolution Grid 1",
    origin=Point3(100, 200, 0),
    size=size,
    cell_sizes_x=cell_sizes_x,
    cell_sizes_y=cell_sizes_y,
    cell_sizes_z=cell_sizes_z,
    cell_data=pd.DataFrame(
        {
            "grade": np.random.rand(num_cells) * 10,
            "density": np.random.rand(num_cells) * 2 + 2,
        }
    ),
    vertex_data=pd.DataFrame(
        {
            "elevation": np.random.rand(num_vertices) * 100,
        }
    ),
)
created_tensor_grid = await Tensor3DGrid.create(manager, tensor_grid)
print(f"Created tensor-3d-grid object: {created_tensor_grid.metadata.url}")
print(f"Grid extent: X={np.sum(cell_sizes_x):.1f}, Y={np.sum(cell_sizes_y):.1f}, Z={np.sum(cell_sizes_z):.1f}")

### Downloading and inspecting a tensor grid

Simply displaying the tensor grid shows a pretty-printed summary. The cell sizes for each axis can be accessed via the `cell_sizes_x`, `cell_sizes_y`, and `cell_sizes_z` properties.

In [ ]:
from evo.objects.typed import object_from_reference

tensor_grid = await object_from_reference(manager, created_tensor_grid.metadata.url)

# Pretty-printed display
tensor_grid

In [ ]:
# View cell and vertex attributes
print("Cell attributes:")
display(tensor_grid.attributes)
print("\nVertex attributes:")
display(tensor_grid.vertex_attributes)

In [ ]:
print(f"Origin: {tensor_grid.origin}")
print(f"Size: {tensor_grid.size}")
print(f"Bounding Box: {tensor_grid.bounding_box}")
print(f"\nCell sizes X: {tensor_grid.cell_sizes_x}")
print(f"Cell sizes Y: {tensor_grid.cell_sizes_y}")
print(f"Cell sizes Z: {tensor_grid.cell_sizes_z}")

In [ ]:
# Get the cell data using to_dataframe()
cell_df = await tensor_grid.to_dataframe()
print(f"Cell data: {cell_df.shape[0]} cells with columns {list(cell_df.columns)}")
cell_df.head()

In [ ]:
# Get vertex data
vertex_df = await tensor_grid.vertices.get_dataframe()
print(f"Vertex data: {vertex_df.shape[0]} vertices with columns {list(vertex_df.columns)}")
vertex_df.head()
